# General Imports

In [1]:
import pandas as pd
import numpy as np
import yaml
import random
import requests
#from tqdm import tqdm
from tqdm.notebook import tqdm

# Using BioPython

In [2]:
import Bio.PDB as bp
from pathlib import Path

In [29]:
def is_het(residue):
    ''' Finds out if a given residue is HET
        
        The residue id is a tuple with 3 elements:
        
        full_id[0] = Hetero flag = "H_"+resname for hets, or "W" for water.
        full_id[1] = Sequence identifier (resnumber)
        full_is[2] = Insertion code (needed when a second residue with same number is
                     inserted in the sequence, e.g., by inserting a new residue).
                     This is NOT the chain ID.
                     
        For example, for a "glucose", the result of residue.id 
        could be something like: "('H_GLC', 100, 'A')"
    
    '''
    res = residue.id[0]
    return res !=" " and res !="W"

In [4]:
# PDB mmCIF file parser
cif_parser = bp.MMCIFParser()

In [5]:
!pwd

/home/seabra/work/li/databases/pdb_db


In [6]:
pdbid = "101M"

mmcif_file = f"./pdb/{pdbid[:2]}/{pdbid.lower()}.cif"
print(f"File: {mmcif_file} exists? {Path(mmcif_file).is_file()}")
struct = cif_parser.get_structure(pdbid, mmcif_file)

File: ./pdb/10/101m.cif exists? True


In [7]:
struct.id

'101M'

In [8]:
struct.header

{'name': 'SPERM WHALE MYOGLOBIN F46V N-BUTYL ISOCYANIDE AT PH 9.0',
 'head': 'OXYGEN TRANSPORT',
 'idcode': '101M',
 'deposition_date': '1997-12-13',
 'structure_method': 'X-RAY DIFFRACTION',
 'resolution': 2.07}

In [9]:
struct.header['resolution']

2.07

In [10]:
struct.child_list

[<Model id=0>]

In [15]:
for chain in struct.get_chains():
    print("Full ID: ", chain.full_id)
    print("ID: ", chain.id)
    print("Level: ", chain.level)

Full ID:  ('101M', 0, 'A')
ID:  A
Level:  C


In [17]:
for chain in struct.get_chains():
    print(chain.xtra)

{}


In [20]:
for residue in struct.get_residues():
    if is_het(residue): print(residue.full_id, residue.resname)

('101M', 0, 'A', ('H_SO4', 157, ' ')) SO4
('101M', 0, 'A', ('H_HEM', 155, ' ')) HEM
('101M', 0, 'A', ('H_NBN', 156, ' ')) NBN


This means we will need to remove the ions as well when making the PDB file. 

In [21]:
def get_hets(structure):
    '''returns a list of HETS in the structure'''
    hets = []
    for residue in structure.get_residues():
        if is_het(residue): hets.append(residue)
    return hets

In [22]:
def select_residue(structure,full_res_id):
    '''Selects a residue given its full ID'''
    _pdbid = full_res_id[0]
    _model = full_res_id[1]
    _chain = full_res_id[2]
    _resnum= full_res_id[3][1]
    return 

In [25]:
hets = get_hets(struct)
hets

[<Residue SO4 het=H_SO4 resseq=157 icode= >,
 <Residue HEM het=H_HEM resseq=155 icode= >,
 <Residue NBN het=H_NBN resseq=156 icode= >]

In [93]:
lig = hets[2]
lig.resname

'NBN'

In [94]:
# The `full_id` includes all information:
# (pdb_code, model, chain, residue_id)
lig.full_id

('X', ('H_NBN', 156, ' '))

In [30]:
# Just the residue_id, which includes:
# (hetero_flag, sequence_number, sequence_modifier (NOT CHAIN))
lig.id

('H_NBN', 156, ' ')

In [31]:
def get_atoms_list(ligand):
    '''Gets a list of the ligand atoms, excluding hydrogens'''
    atom_list = []
    for atom in ligand.get_atoms():
        if atom.element != 'H': atom_list.append(atom)
    return atom_list

In [32]:
lig.child_list

[<Atom C>, <Atom N>, <Atom C1>, <Atom C2>, <Atom C3>, <Atom C4>]

In [36]:
# Creates a neighbor search object containing ONLY the atoms of the ligand,
# with a radius of 10A. Notice it only returns atoms from the ligand itself!
neighbor_searcher = bp.NeighborSearch(get_atoms_list(lig), bucket_size=10)

In [37]:
neighbor_searcher.search_all(radius=10,level='A')

[(<Atom C>, <Atom N>),
 (<Atom C>, <Atom C1>),
 (<Atom C>, <Atom C2>),
 (<Atom C>, <Atom C3>),
 (<Atom C>, <Atom C4>),
 (<Atom N>, <Atom C1>),
 (<Atom N>, <Atom C2>),
 (<Atom N>, <Atom C3>),
 (<Atom N>, <Atom C4>),
 (<Atom C1>, <Atom C2>),
 (<Atom C1>, <Atom C3>),
 (<Atom C1>, <Atom C4>),
 (<Atom C2>, <Atom C3>),
 (<Atom C2>, <Atom C4>),
 (<Atom C3>, <Atom C4>)]

In [39]:
# Gets a list of *all* atoms in the structure
atoms  = bp.Selection.unfold_entities(struct, 'A')
len(atoms)

1413

In [40]:
# Now we can create a NeighborSearch object with all the atoms in the 
# structure, where it finds all neighbors within 10A of each other:
neighbor_searcher = bp.NeighborSearch(atoms, bucket_size=10)

In [46]:
# This gets a lilst of all residues within 5 angstroms of the ligand
closest = set()
for atom in get_atoms_list(lig):
    print(atom, atom.coord)
    #print(neighbor_searcher.search(atom.coord,5, level='R'))
    closest = closest | set(neighbor_searcher.search(atom.coord,5, level='R'))
    print(len(closest))
    
print(closest)

<Atom C> [36.437  5.629 11.224]
5
<Atom N> [36.674  6.332 11.97 ]
6
<Atom C1> [37.691  6.621 13.103]
7
<Atom C2> [38.602  5.446 13.328]
8
<Atom C3> [39.713  5.441 12.363]
9
<Atom C4> [40.675  4.358 12.79 ]
9
{<Residue HIS het=  resseq=64 icode= >, <Residue VAL het=  resseq=68 icode= >, <Residue NBN het=H_NBN resseq=156 icode= >, <Residue HEM het=H_HEM resseq=155 icode= >, <Residue LEU het=  resseq=29 icode= >, <Residue THR het=  resseq=67 icode= >, <Residue PHE het=  resseq=43 icode= >, <Residue HIS het=  resseq=93 icode= >, <Residue HOH het=W resseq=220 icode= >}


In [48]:
for res in struct.get_residues():
    resnum = res.get_id()[1]
    if resnum == 93:
        print(get_atoms_list(res))

[<Atom N>, <Atom CA>, <Atom C>, <Atom O>, <Atom CB>, <Atom CG>, <Atom ND1>, <Atom CD2>, <Atom CE1>, <Atom NE2>]


In [49]:
# This gets a list of all chains with residues within 5 angstroms of the ligand
closest = set()
for atom in get_atoms_list(lig):
    print(atom, atom.coord)
    #print(neighbor_searcher.search(atom.coord,5, level='R'))
    closest = closest | set(neighbor_searcher.search(atom.coord,5, level='C'))
    print(len(closest))
    
print(closest)

<Atom C> [36.437  5.629 11.224]
1
<Atom N> [36.674  6.332 11.97 ]
1
<Atom C1> [37.691  6.621 13.103]
1
<Atom C2> [38.602  5.446 13.328]
1
<Atom C3> [39.713  5.441 12.363]
1
<Atom C4> [40.675  4.358 12.79 ]
1
{<Chain id=A>}


# Testing a workflow

In [106]:
# Read PDB data
pdb_data = pd.read_pickle('pdb_no_cofactors.pkl').sample(5)

In [107]:
pdb_data

,PDB_ID,RESOLUTION,POLY_NAME,POLY_TYPE,POLY_CHAINS,IS_MEMBRANE,LIG_ID,LIG_CHAINS,LIG_NAME,LIG_SMILES,LIG_TYPE,LIG_MW,INVESTIGN?
75198,7KJQ,1.35,"Transcriptional regulator, MarR family",Protein,['A'],False,WOM,['A'],Picloram,c1(c(c(nc(c1Cl)Cl)C(=O)O)Cl)N,non-polymer,241.459,0.0
26009,6LG7,1.83,Bromodomain-containing protein 4,Protein,['A'],False,ECF,['A'],2-azanyl-6-fluoranyl-4-imidazol-1-yl-quinolin-...,c1cn(cn1)c2cc(nc3c2cc(cc3O)F)N,non-polymer,244.224,1.0
54011,2ZA0,1.70,Glyoxalase I,Protein,"['B', 'A']",False,MGI,['A'],"methyl 4-(2,3-dihydroxy-5-methylphenoxy)-2-hyd...",Cc1cc(c(c(c1)Oc2cc(c(c(c2)O)C(=O)OC)C)O)O,non-polymer,304.295,0.0
76491,7S45,1.20,N-acetyltransferase,Protein,['A'],False,TYD,['A'],THYMIDINE-5'-DIPHOSPHATE,CC1=CN(C(=O)NC1=O)[C@H]2C[C@@H]([C@H](O2)CO[P@...,non-polymer,402.188,0.0
79292,1TLG,2.20,POLYANDROCARPA LECTIN,Protein,"['B', 'A']",False,GAL,"['B', 'A']",beta-D-galactopyranose,C([C@@H]1[C@@H]([C@@H]([C@H]([C@@H](O1)O)O)O)O)O,"D-saccharide, beta linking",180.156,0.0


In [112]:
len(pdb_data)

5

In [108]:
pdb_data.columns

Index(['PDB_ID', 'RESOLUTION', 'POLY_NAME', 'POLY_TYPE', 'POLY_CHAINS',
       'IS_MEMBRANE', 'LIG_ID', 'LIG_CHAINS', 'LIG_NAME', 'LIG_SMILES',
       'LIG_TYPE', 'LIG_MW', 'INVESTIGN?'],
      dtype='object')

In [111]:
!ls pdb/7K/7kjq.cif

pdb/7K/7kjq.cif


In [114]:
!pwd

/home/seabra/work/li/databases/pdb_db


In [119]:
!ls --color

3QJG_chains_ABCD.pdb  obsolete                   processed_pdbs
build_df_parallel.py  pdb                        query.log
build_df.py           pdb_full_filtered.pkl      SAVE_pdb_info_df.pkl
cofactor_data.csv     pdb_info_df.pkl            small_lig_data.csv
df-2.log              pdb_no_cofactors.pkl       TEMP_pdb_info_df.pkl
df.log                pdbquery.ipynb             treat_pdb_files.ipynb
download_pdb.py       pdbquery_scratchpad.ipynb  treat_pdbquery.ipynb
failed.pkl            pdb_w_cofactors.pkl        Untitled.ipynb


In [147]:
def get_heavy_atoms_list(residue):
    '''Gets a list of the residue atoms, excluding hydrogens'''
    atom_list = []
    for atom in residue.get_atoms():
        if atom.element != 'H': atom_list.append(atom)
    return atom_list

In [133]:
def get_ligands(structure, ligand):
    '''returns a list of ligand residues in the structure'''
    ligands = []
    for residue in structure.get_residues():
        if residue.resname == ligand: ligands.append(residue)
    return ligands

In [191]:
class AtomsToPrintSelector(bp.Select):
    """Class to define the selection atoms"""
    def __init__(self, chains):
        self.chains   = [chain.id for chain in chains]
        self.nonligand_hets = ['WAT','HOH']
        return
    
    def accept_chain(self, chain):
        accept = False
        if chain.id in self.chains:
            accept = True
        return accept
    
    def accept_residue(self, residue):
        # Cleans undesirable HETs 
        accept = True
        res = residue.id[0]
        if res !=" " and residue.resname in self.nonligand_hets:
            accept = False
        return accept
        
    def accept_atom(self, atom):
        # Eliminate alternate positions
        accept = False
        if (not atom.is_disordered()) or atom.get_altloc() == "A":
            atom.set_altloc(" ")  # Eliminate alt location ID before output.
            accept = True
        return accept

In [128]:
orig_root = Path("pdb")
dest_root = Path('processed_pdbs')
dest_root.mkdir(exist_ok=True)

We need to iterate the dataframe row-by-row

In [192]:
cif_parser = bp.MMCIFParser(QUIET=True)
io = bp.PDBIO()
cutoff_dist = 6
for idx, row in tqdm(pdb_data.iterrows()):
    
    pdb_id = row.PDB_ID
    lig_id = row.LIG_ID
    
    orig_dir = Path(orig_root, pdb_id[:2])
    dest_dir = Path(dest_root, pdb_id[:2])
    dest_dir.mkdir(exist_ok=True)
    
    orig_file = Path(orig_dir, f"{pdb_id.lower()}.cif")
    dest_file = str(Path(dest_dir, f"{pdb_id.lower()}.pdb"))
    
    if orig_file.exists():
        print("\nProcessing file: ", orig_file)
        print(f"  PDBID: {pdb_id}    Ligand: {lig_id} ")
    else:
        print("Could not find file: ", orig_file)
        break
        
    # Read the PDB file
    struct = cif_parser.get_structure(pdb_id, orig_file)[0]

    # Sometimes, the same ligand appears more than once in the same chain.
    # Here we choose to eliminate these repetitions.
    ligs = get_ligands(struct, lig_id)
    print("  Found ligands:  ", ligs)
    lig = ligs[0]
    print("  Keeping ligand: ", lig.full_id)
    
    for ligand in ligs:
        orig_chain = ligand.get_parent()
        ligand.detach_parent()
        orig_chain.detach_child(ligand.get_id())
 
    # For convenience, lets make all ligants to be in chain "Y"
    chain_Y = bp.Chain.Chain('Y')
    chain_Y.add(lig)
    lig.set_parent(chain_Y)
    struct.add(chain_Y)
    print("  Converted to: ", lig.full_id)
    
    # Now we need to find all chains with any atoms within cutoff_dist from the ligand
    neighbor_searcher = bp.NeighborSearch(bp.Selection.unfold_entities(struct, 'A'), bucket_size=10)
    
    
    closest = set()
    for atom in get_heavy_atoms_list(lig):
        #print(neighbor_searcher.search(atom.coord,5, level='R'))
        closest = closest | set(neighbor_searcher.search(atom.coord,cutoff_dist, level='C'))
    
    print(f"  Found chains within {cutoff_dist} Angstroms of ligand atoms: {closest}")
    
    # now, save only the atoms from these chains to a new PDB file
    selector = AtomsToPrintSelector(closest)
    io.set_structure(struct)
    io.save(dest_file,select=selector,
            write_end=True, preserve_atom_numbering=False)
    

0it [00:00, ?it/s]


Processing file:  pdb/7K/7kjq.cif
  PDBID: 7KJQ    Ligand: WOM 
  Found ligands:   [<Residue WOM het=H_WOM resseq=201 icode= >]
  Keeping ligand:  ('7KJQ', 0, 'A', ('H_WOM', 201, ' '))
  Converted to:  ('7KJQ', 0, 'Y', ('H_WOM', 201, ' '))
  Found chains within 6 Angstroms of ligand atoms: {<Chain id=Y>, <Chain id=A>}

Processing file:  pdb/6L/6lg7.cif
  PDBID: 6LG7    Ligand: ECF 
  Found ligands:   [<Residue ECF het=H_ECF resseq=201 icode= >]
  Keeping ligand:  ('6LG7', 0, 'A', ('H_ECF', 201, ' '))
  Converted to:  ('6LG7', 0, 'Y', ('H_ECF', 201, ' '))
  Found chains within 6 Angstroms of ligand atoms: {<Chain id=Y>, <Chain id=A>}

Processing file:  pdb/2Z/2za0.cif
  PDBID: 2ZA0    Ligand: MGI 
  Found ligands:   [<Residue MGI het=H_MGI resseq=300 icode= >, <Residue MGI het=H_MGI resseq=400 icode= >]
  Keeping ligand:  ('2ZA0', 0, 'A', ('H_MGI', 300, ' '))
  Converted to:  ('2ZA0', 0, 'Y', ('H_MGI', 300, ' '))
  Found chains within 6 Angstroms of ligand atoms: {<Chain id=Y>, <Chain 

In [163]:
type(chain_Y.id)

str

In [172]:
chain_Y.detach_child

[]

In [182]:
lig.get_id()

('H_WOM', 201, ' ')